In [1]:
!pip install control slycot

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 16.3 MB/s eta 0:00:00a 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 455.1/455.1 kB 5.6 MB/s eta 0:00:00:00:0100:01
  Created wheel for slycot: filename=slycot-0.5.4-cp38-cp38-macosx_12_0_arm64.whl size=1854075 sha256=80c097cde8100b9a74a13855f6ec7edbd3fad0828b16f3f0774e0a3bf83d277a
  Stored in directory: /Users/atsu/Library/Caches/pip/wheels/85/1a/04/2fd052437f00cfc2c638f221e2812eb53d771f93f78ac514b3
Successfully built slycot


In [5]:
import matplotlib.pyplot as plt
import numpy as np
from control.matlab import *
from ipywidgets import FloatSlider, interact

class pid_controller:
    def __init__(self, kp, ki, kd, dt, setpoint=1.0):
        self.kp = kp
        self.ki = ki
        self.kd = kd
        self.dt = dt
        self.setpoint = setpoint
        self.error_sum = 0
        self.last_error = 0

    def update(self, y_val):
        error = self.setpoint - y_val
        self.error_sum += error * self.dt
        error_derivative = (error - self.last_error) / self.dt
        self.last_error = error

        return self.kp * error + self.ki * self.error_sum + self.kd * error_derivative


def func1(kp, ki, kd, dt):
    """微分方程式を解いて y(t) を求める方法

    PID 制御システムの伝達関数の逆ラプラス変換を数学的に求めてプロットする。

    Args:
        kp: 比例ゲイン
        ki: 積分ゲイン
        kd: 微分ゲイン
        dt: プロット間隔 (秒)
    """

    # 時間軸 表示範囲 (0 から 60 まで dt 秒刻み)
    t = np.arange(0, 60, dt)

    zeta2 = kp * kp - 4 * ki * (kd + 1)

    # zeta2 の正負によって場合分け
    if zeta2 > 0:
        zeta = np.sqrt(zeta2)
        alpha = 0.5 * kp / (kd + 1)
        omega = 0.5 * zeta / (kd + 1)
        y = 1 - (np.exp(-alpha * t) * (np.cosh(omega * t) - (kp / zeta) * np.sinh(omega * t))) / (kd + 1)
    elif zeta2 < 0:
        zeta = np.sqrt(-zeta2)
        alpha = 0.5 * kp / (kd + 1)
        omega = 0.5 * zeta / (kd + 1)
        y = 1 - (np.exp(-alpha * t) * (np.cos(omega * t) - (kp / zeta) * np.sin(omega * t))) / (kd + 1)
    else:
        alpha = 0.5 * kp / (kd + 1)
        y = 1 - (np.exp(-alpha * t) * (1 - alpha * t)) / (kd + 1)

    return y, t


def func2(kp, ki, kd, dt):
    """Python-Control を用いて y(t) を求める方法

    Args:
        kp: 比例ゲイン
        ki: 積分ゲイン
        kd: 微分ゲイン
        dt: プロット間隔 (秒)
    """

    # 時間軸 表示範囲 (0 から 60 まで dt 秒刻み)
    t = np.arange(0, 60, dt)

    # PID 制御の伝達関数: K(s) = kp + ki/s + kd*s
    num = [kd, kp, ki]
    den = [1, 0]
    K = tf(num, den)
    # print(K)

    # 制御対象の伝達関数: G(s) = 1/s
    num = [1]
    den = [1, 0]
    G = tf(num, den)
    # print(G)

    # K,G を結合してフィードバック
    sys = feedback(K * G, 1)
    # print(sys)

    # ステップ入力の応答を求める。
    y, t = step(sys, t)

    return y, t


def func3(kp, ki, kd, dt):
    """自作の PID 制御関数を使って y(t) を求める方法

    Args:
        kp: 比例ゲイン
        ki: 積分ゲイン
        kd: 微分ゲイン
        dt: プロット間隔 (秒)
    """

    # 時間軸 表示範囲 (0 から 60 まで dt 秒刻み)
    t = np.arange(0, 60, dt)

    # y(t)
    y = []

    # ポイント数
    npoints = len(t)

    # PID 制御器のシミュレータ
    controller = pid_controller(kp, ki, kd, dt)

    # 現時点での u(t) および y(t) の値
    u_val = 0
    y_val = 0

    for i in range(0, npoints):
        y.append(y_val)
        u_val = controller.update(y_val)
        y_val += u_val * dt

    return y, t


# PID パラメータを調節するスライダーを表示
@interact(
    kp=FloatSlider(min=0, max=1, step=0.01, value=0.3),
    ki=FloatSlider(min=0, max=1, step=0.01, value=0.2),
    kd=FloatSlider(min=0, max=1, step=0.01, value=0.1),
)
def func(kp, ki, kd):
    dt = 0.1

    y1, t1 = func1(kp, ki, kd, dt)
    plt.plot(t1, y1, label="y1")

    y2, t2 = func2(kp, ki, kd, dt)
    plt.plot(t2, y2, linestyle="--", label="y2")

    y3, t3 = func3(kp, ki, kd, dt)
    plt.plot(t3, y3, label="y3")

    plt.xlabel("t [sec]")
    plt.ylabel("y")
    plt.title("PID Control")
    plt.legend()
    plt.show()

interactive(children=(FloatSlider(value=0.3, description='kp', max=1.0, step=0.01), FloatSlider(value=0.2, des…